# Experiment Notebook: PCV Management

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
from operator import lt, gt

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

In [ ]:
parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
    "rebalancing_period": [int(365 / 4)],
    "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
    "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
}

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

# Analysis 1: ...

{Analysis Description}

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

In [ ]:
df.query('subset==0').plot(
    x='timestep',
    y=['total_volatile_asset_pcv', 'total_stable_asset_pcv', 'total_pcv'],
    title="Policy decreases stable PCV ratio to a target of 20%"
)

In [ ]:
df.query('subset==1').plot(
    x='timestep',
    y=['total_volatile_asset_pcv', 'total_stable_asset_pcv', 'total_pcv'],
    title="Policy increases stable PCV ratio to a target of 50%"
)

In [ ]:
df.query('subset==0')['stable_pcv_ratio'].plot(
    title="Policy decreases stable PCV ratio to a target of 20%"
)

In [ ]:
df.query('subset==1')['stable_pcv_ratio'].plot(
    title="Policy increases stable PCV ratio to a target of 50%"
)

### PCV Constituents

<b>Note:</b> Filter by appropriate subset here

In [ ]:
df.plot(x='timestep', y='volatile_deposit_yield_bearing_balance', color='subset')

In [ ]:
df.plot(x='timestep', y='volatile_deposit_idle_balance', color='subset')

In [ ]:
df.plot(x='timestep', y='stable_deposit_yield_bearing_balance', color='subset')

In [ ]:
df.plot(x='timestep', y='stable_deposit_idle_balance', color='subset')

In [ ]:
df.plot(x='timestep', y='total_stable_asset_pcv_balance', color='subset')

In [ ]:
df.plot(x='timestep', y='total_volatile_asset_pcv_balance', color='subset')

### Descriptive stats

In [ ]:
df.plot(x='timestep', y='collateralization_ratio', color='subset')